# Routing

In [ ]:
AUTHENTICATE_ROUTE = "authenticate"
BLOOD_PRESSURE_ROUTE = "bloodpressure"
BLOOD_GLUCOSE_ROUTE = "bloodglucose"
PERSON_ROUTE = "person"
WEIGHT_ROUTE = "weight"

ROUTES = {
    AUTHENTICATE_ROUTE: "/users/authenticate",
    BLOOD_PRESSURE_ROUTE: "/bloodpressuremeasurement",
    BLOOD_GLUCOSE_ROUTE: "/bloodglucosemeasurement",
    PERSON_ROUTE: "/person",
    WEIGHT_ROUTE: "/weightmeasurement"
}

# Utility Methods

In [ ]:
from urllib.parse import quote

def encode_date(date):
    date_string = date.strftime("%Y-%m-%d %H:%M:%S")
    encoded = quote(date_string)
    return encoded

# Authentication

In [ ]:
import requests

def authenticate(url, username, password):
    payload = { "UserName": username, "Password": password }
    endpoint = f"{url}{ROUTES[AUTHENTICATE_ROUTE]}"
    response = requests.post(endpoint, json=payload)
    response.raise_for_status()
    return response.text

# Person Retrieval and Identification

In [ ]:
import requests
import pandas as pd

def list_people(url, token):
    headers = { "Authorization": f"Bearer {token}"}
    endpoint = f"{url}{ROUTES[PERSON_ROUTE]}"
    response = requests.get(endpoint, headers=headers)
    response.raise_for_status()
    data = response.json()
    return pd.DataFrame(data)

def get_person(url, token, firstnames, surname):
    people = list_people(url, token)
    person = people[
        (people["firstNames"].str.casefold() == firstnames.casefold()) &
        (people["surname"].str.casefold() == surname.casefold())
    ]
    return person

def get_person_id(url, token, firstnames, surname):
    person = get_person(url, token, firstnames, surname)
    return person.loc[0, "id"]

# Blood Pressure Data Retrieval

In [ ]:
import requests
import pandas as pd

def get_daily_blood_pressure_measurements(url, token, person_id, start, end):
    # URL encode the dates
    encoded_start = encode_date(start)
    encoded_end = encode_date(end)

    # Send the request to the API
    headers = { "Authorization": f"Bearer {token}"}
    endpoint = f"{url}{ROUTES[BLOOD_PRESSURE_ROUTE]}/dailyaverage/{person_id}/{encoded_start}/{encoded_end}"
    response = requests.get(endpoint, headers=headers)
    response.raise_for_status()

    # Extract the response JSON and convert to a dataframe then make sure the date column
    # contains dates rather than string representations of dates
    data = response.json()
    df = pd.DataFrame(data)
    df["date"] = pd.to_datetime(df["date"])

    return df


# Weight Measurement Data Retrieval

In [ ]:
import requests
import pandas as pd

def get_weight_measurements(url, token, person_id, start, end):
    # URL encode the dates
    encoded_start = encode_date(start)
    encoded_end = encode_date(end)

    # Send the request to the API
    headers = { "Authorization": f"Bearer {token}"}
    endpoint = f"{url}{ROUTES[WEIGHT_ROUTE]}/{person_id}/{encoded_start}/{encoded_end}"
    response = requests.get(endpoint, headers=headers)
    response.raise_for_status()

    # Extract the response JSON and convert to a dataframe then make sure the date column
    # contains dates rather than string representations of dates
    data = response.json()
    df = pd.DataFrame(data)
    df["date"] = pd.to_datetime(df["date"])

    return df


# Blood Glucose Data Retrieval

In [ ]:
import requests
import pandas as pd

def get_blood_glucose_measurements(url, token, person_id, start, end):
    # URL encode the dates
    encoded_start = encode_date(start)
    encoded_end = encode_date(end)

    # Send the request to the API
    headers = { "Authorization": f"Bearer {token}"}
    endpoint = f"{url}{ROUTES[BLOOD_GLUCOSE_ROUTE]}/{person_id}/{encoded_start}/{encoded_end}"
    response = requests.get(endpoint, headers=headers)
    response.raise_for_status()

    # Extract the response JSON and convert to a dataframe then make sure the date column
    # contains dates rather than string representations of dates
    data = response.json()
    df = pd.DataFrame(data)
    df["date"] = pd.to_datetime(df["date"])

    return df
